In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType , DateType 
from pyspark.sql.functions import sum, col,avg,desc


In [2]:
scSpark = SparkSession.builder.appName("Spark Example").getOrCreate()

23/05/16 16:42:46 WARN Utils: Your hostname, all-MS-7D35 resolves to a loopback address: 127.0.1.1; using 192.168.1.134 instead (on interface enp2s0)
23/05/16 16:42:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/16 16:42:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
transaction = scSpark.read.csv("transactions_*.csv", header=True, inferSchema=True)
customer = scSpark.read.csv("customers.csv", header=True, inferSchema=True)
products = scSpark.read.csv("products.csv", header=True, inferSchema=True)

# Define join conditions
join_condition1 = transaction["CustomerId"] == customer["CustomerId"]

join_condition2 = transaction["ProductId"] == products["ProductId"]

# Perform joins
joined_df = transaction.join(customer, join_condition1, "left") \
                      .join(products, join_condition2, "left")

# Show the resulting joined data frame
joined_df.show()

+-------+-------------+----------+---------+--------+-------------------+----------+--------------------+--------------------+---------+-------------+--------+---------+
|StoreId|TransactionId|CustomerId|ProductId|Quantity|    TransactionTime|CustomerId|                Name|               Email|ProductId|         Name|Category|UnitPrice|
+-------+-------------+----------+---------+--------+-------------------+----------+--------------------+--------------------+---------+-------------+--------+---------+
|      3|          454|        35|        3|       3|2022-12-23 17:36:11|        35|      Dwayne Johnson|dwayne.johnson@gm...|        3|  Blue Shorts|  Shorts|   118.88|
|      3|          524|        37|        9|      11|2022-12-23 22:02:51|        37|       Brittany Holt|brittany.holt@exa...|        9|Green Sandals|   Shoes|   137.53|
|      3|          562|         4|        3|       4|2022-12-23 02:51:50|         4|       Alevtin Paska|alevtin.paska@exa...|        3|  Blue Shorts|

In [4]:
join_df = joined_df.withColumn("total price", col("Quantity") * col("UnitPrice"))
join_df.show()

+-------+-------------+----------+---------+--------+-------------------+----------+--------------------+--------------------+---------+-------------+--------+---------+------------------+
|StoreId|TransactionId|CustomerId|ProductId|Quantity|    TransactionTime|CustomerId|                Name|               Email|ProductId|         Name|Category|UnitPrice|       total price|
+-------+-------------+----------+---------+--------+-------------------+----------+--------------------+--------------------+---------+-------------+--------+---------+------------------+
|      3|          454|        35|        3|       3|2022-12-23 17:36:11|        35|      Dwayne Johnson|dwayne.johnson@gm...|        3|  Blue Shorts|  Shorts|   118.88|            356.64|
|      3|          524|        37|        9|      11|2022-12-23 22:02:51|        37|       Brittany Holt|brittany.holt@exa...|        9|Green Sandals|   Shoes|   137.53|           1512.83|
|      3|          562|         4|        3|       4|20

In [5]:


# Convert the "Quantity" column to the appropriate data type
join_df = join_df.withColumn("total price", col("total price").cast("float"))
join_df= join_df.withColumn("TransactionTime", join_df["TransactionTime"].cast(DateType()))

store1_sales_df = join_df.filter(join_df["StoreId"] == 1)

# Calculate the daily total sales
daily_total_sales_store1 = store1_sales_df.groupBy("TransactionTime").agg(sum("total price").alias("TotalSales"))

# Show the results
daily_total_sales_store1.show()

+---------------+-----------------+
|TransactionTime|       TotalSales|
+---------------+-----------------+
|     2022-12-23|41264.00012207031|
+---------------+-----------------+



In [6]:
store2_sales_df = join_df.filter(join_df["StoreId"] == 2)

# Calculate the mean sales
mean_sales_store2 = store2_sales_df.select(avg("total price").alias("MeanSales"))

# Show the results
mean_sales_store2.show()

+-----------------+
|        MeanSales|
+-----------------+
|513.4598035625382|
+-----------------+



In [7]:
join_df.show()

+-------+-------------+----------+---------+--------+---------------+----------+--------------------+--------------------+---------+-------------+--------+---------+-----------+
|StoreId|TransactionId|CustomerId|ProductId|Quantity|TransactionTime|CustomerId|                Name|               Email|ProductId|         Name|Category|UnitPrice|total price|
+-------+-------------+----------+---------+--------+---------------+----------+--------------------+--------------------+---------+-------------+--------+---------+-----------+
|      3|          454|        35|        3|       3|     2022-12-23|        35|      Dwayne Johnson|dwayne.johnson@gm...|        3|  Blue Shorts|  Shorts|   118.88|     356.64|
|      3|          524|        37|        9|      11|     2022-12-23|        37|       Brittany Holt|brittany.holt@exa...|        9|Green Sandals|   Shoes|   137.53|    1512.83|
|      3|          562|         4|        3|       4|     2022-12-23|         4|       Alevtin Paska|alevtin.p

In [8]:
total_sales_per_customer = join_df.groupBy("Email").agg(sum("total price").alias("TotalSales"))
# total_sales_per_customer.sort(desc("TotalSales"))
# total_sales_per_customer.show()
a= total_sales_per_customer.sort(desc("TotalSales"))
b=a.show(1)
# top_customer = total_sales_per_customer.first()

# Sort the total sales in descending order
# sorted_sales = total_sales_per_customer.sort(desc("TotalSales"))

# # Retrieve the customer with the highestotal_salest total sales
# top_customer = sorted_sales.first()

# # Retrieve the email associated with the top customer
# email_of_top_customer = clients_df.filter(clients_df["CustomerId"] == top_customer["CustomerId"]).select("Email").first()[0]

# # Print the email of the customer who spent the most
print("Email of the client who spent the most:", b)

+--------------------+------------------+
|               Email|        TotalSales|
+--------------------+------------------+
|dwayne.johnson@gm...|10653.080017089844|
+--------------------+------------------+
only showing top 1 row

Email of the client who spent the most: None


In [9]:
join_df = join_df.drop(products["ProductId"])

# Show the updated DataFrame
join_df.show()

+-------+-------------+----------+---------+--------+---------------+----------+--------------------+--------------------+-------------+--------+---------+-----------+
|StoreId|TransactionId|CustomerId|ProductId|Quantity|TransactionTime|CustomerId|                Name|               Email|         Name|Category|UnitPrice|total price|
+-------+-------------+----------+---------+--------+---------------+----------+--------------------+--------------------+-------------+--------+---------+-----------+
|      3|          454|        35|        3|       3|     2022-12-23|        35|      Dwayne Johnson|dwayne.johnson@gm...|  Blue Shorts|  Shorts|   118.88|     356.64|
|      3|          524|        37|        9|      11|     2022-12-23|        37|       Brittany Holt|brittany.holt@exa...|Green Sandals|   Shoes|   137.53|    1512.83|
|      3|          562|         4|        3|       4|     2022-12-23|         4|       Alevtin Paska|alevtin.paska@exa...|  Blue Shorts|  Shorts|   118.88|     

In [10]:
# Assuming you have a joined DataFrame named 'joined_df' with the required columns

# Group by ProductId and count the occurrences
product_count_df = join_df.groupBy("ProductId").agg(sum("Quantity").alias("totalQuantity"))

# Order by count in descending order
ordered_product_count_df = product_count_df.orderBy("totalQuantity", ascending=False)

# Select the top 5 most frequently bought products
top_5_products = ordered_product_count_df.limit(5)

# Show the resulting DataFrame
top_5_products.show()

+---------+-------------+
|ProductId|totalQuantity|
+---------+-------------+
|       14|           82|
|       24|           77|
|       15|           76|
|        5|           75|
|       19|           74|
+---------+-------------+



In [11]:
join_condition = top_5_products["ProductId"] == products["ProductId"]
joined_df = top_5_products.join(products,join_condition)
joined_df.orderBy("totalQuantity", ascending=False).show(5)

+---------+-------------+---------+-------------+--------+---------+
|ProductId|totalQuantity|ProductId|         Name|Category|UnitPrice|
+---------+-------------+---------+-------------+--------+---------+
|       14|           82|       14|  Red t-shirt|T-Shirts|   121.58|
|       24|           77|       24|   Blue Jeans|   Pants|    173.1|
|       15|           76|       15|White t-shirt|T-Shirts|   131.13|
|        5|           75|        5| Black Shorts|  Shorts|    74.58|
|       19|           74|       19| Green jacket| Jackets|   223.69|
+---------+-------------+---------+-------------+--------+---------+

